In [86]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 21: Dirac Dice ---</h2><p>There's not much to do as you slowly descend to the bottom of the ocean. The submarine computer <span title="A STRANGE GAME.">challenges you to a nice game</span> of <em>Dirac Dice</em>.</p>
<p>This game consists of a single <a href="https://en.wikipedia.org/wiki/Dice" target="_blank">die</a>, two <a href="https://en.wikipedia.org/wiki/Glossary_of_board_games#piece" target="_blank">pawns</a>, and a game board with a circular track containing ten spaces marked <code>1</code> through <code>10</code> clockwise. Each player's <em>starting space</em> is chosen randomly (your puzzle input). Player 1 goes first.</p>
<p>Players take turns moving. On each player's turn, the player rolls the die <em>three times</em> and adds up the results. Then, the player moves their pawn that many times <em>forward</em> around the track (that is, moving clockwise on spaces in order of increasing value, wrapping back around to <code>1</code> after <code>10</code>). So, if a player is on space <code>7</code> and they roll <code>2</code>, <code>2</code>, and <code>1</code>, they would move forward 5 times, to spaces <code>8</code>, <code>9</code>, <code>10</code>, <code>1</code>, and finally stopping on <code>2</code>.</p>
<p>After each player moves, they increase their <em>score</em> by the value of the space their pawn stopped on. Players' scores start at <code>0</code>. So, if the first player starts on space <code>7</code> and rolls a total of <code>5</code>, they would stop on space <code>2</code> and add <code>2</code> to their score (for a total score of <code>2</code>). The game immediately ends as a win for any player whose score reaches <em>at least <code>1000</code></em>.</p>
<p>Since the first game is a practice game, the submarine opens a compartment labeled <em>deterministic dice</em> and a 100-sided die falls out. This die always rolls <code>1</code> first, then <code>2</code>, then <code>3</code>, and so on up to <code>100</code>, after which it starts over at <code>1</code> again. Play using this die.</p>
<p>For example, given these starting positions:</p>
<pre><code>Player 1 starting position: 4
Player 2 starting position: 8
</code></pre>
<p>This is how the game would go:</p>
<ul>
<li>Player 1 rolls <code>1</code>+<code>2</code>+<code>3</code> and moves to space <code>10</code> for a total score of <code>10</code>.</li>
<li>Player 2 rolls <code>4</code>+<code>5</code>+<code>6</code> and moves to space <code>3</code> for a total score of <code>3</code>.</li>
<li>Player 1 rolls <code>7</code>+<code>8</code>+<code>9</code> and moves to space <code>4</code> for a total score of <code>14</code>.</li>
<li>Player 2 rolls <code>10</code>+<code>11</code>+<code>12</code> and moves to space <code>6</code> for a total score of <code>9</code>.</li>
<li>Player 1 rolls <code>13</code>+<code>14</code>+<code>15</code> and moves to space <code>6</code> for a total score of <code>20</code>.</li>
<li>Player 2 rolls <code>16</code>+<code>17</code>+<code>18</code> and moves to space <code>7</code> for a total score of <code>16</code>.</li>
<li>Player 1 rolls <code>19</code>+<code>20</code>+<code>21</code> and moves to space <code>6</code> for a total score of <code>26</code>.</li>
<li>Player 2 rolls <code>22</code>+<code>23</code>+<code>24</code> and moves to space <code>6</code> for a total score of <code>22</code>.</li>
</ul>
<p>...after many turns...</p>
<ul>
<li>Player 2 rolls <code>82</code>+<code>83</code>+<code>84</code> and moves to space <code>6</code> for a total score of <code>742</code>.</li>
<li>Player 1 rolls <code>85</code>+<code>86</code>+<code>87</code> and moves to space <code>4</code> for a total score of <code>990</code>.</li>
<li>Player 2 rolls <code>88</code>+<code>89</code>+<code>90</code> and moves to space <code>3</code> for a total score of <code>745</code>.</li>
<li>Player 1 rolls <code>91</code>+<code>92</code>+<code>93</code> and moves to space <code>10</code> for a final score, <code>1000</code>.</li>
</ul>
<p>Since player 1 has at least <code>1000</code> points, player 1 wins and the game ends. At this point, the losing player had <code>745</code> points and the die had been rolled a total of <code>993</code> times; <code>745 * 993 = <em>739785</em></code>.</p>
<p>Play a practice game using the deterministic 100-sided die. The moment either player wins, <em>what do you get if you multiply the score of the losing player by the number of times the die was rolled during the game?</em></p>
</article>


In [87]:
from collections.abc import Iterator
from itertools import cycle

from more_itertools import take


class DiracDice:
    def __init__(self, player_1: int, player_2: int, die: Iterator[int]) -> None:
        self.players = [player_1 - 1, player_2 - 1]
        self.scores = [0, 0]
        self.players_turn = 0
        self.die = die
        self.turns = 0

    def turn(self, verbose: bool = False) -> bool:
        player = self.players_turn
        self.players_turn ^= 1
        rolls = list(take(3, self.die)) if verbose else take(3, self.die)
        self.players[player] = (self.players[player] + sum(rolls)) % 10
        self.scores[player] += self.players[player] + 1
        self.turns += 1
        if verbose:
            print(
                f"Player {player + 1} rolls {'+'.join(str(r) for r in rolls)} and moves to space {self.players[player] + 1} for a total score of {self.scores[player]}."
            )
        return self.scores[player] >= 1_000

    def __repr__(self):
        return "\n".join(
            (
                f"Player 1: space: { self.players[0] + 1 }, score: { self.scores[0] }.",
                f"Player 2: space: { self.players[1] + 1 }, score: { self.scores[1] }.",
                f"Turns :{ self.turns } played.",
            )
        )

    @classmethod
    def practice(cls, player_1: int, player_2: int, verbose=False):
        die = cycle(range(1, 101))
        dc = DiracDice(player_1, player_2, die)
        if verbose:
            print(dc)
            print()

        while not dc.turn(verbose):
            pass

        return dc.scores[dc.players_turn] * 3 * dc.turns


# Player 1 starting position: 4
# Player 2 starting position: 8
assert DiracDice.practice(4, 8) == 739785

In [88]:
# Player 1 starting position: 6
# Player 2 starting position: 7

print(f"Part I: { DiracDice.practice(6, 7) }")

Part I: 921585


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>921585</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Now that you're warmed up, it's time to play the real game.</p>
<p>A second compartment opens, this time labeled <em>Dirac dice</em>. Out of it falls a single three-sided die.</p>
<p>As you experiment with the die, you feel a little strange. An informational brochure in the compartment explains that this is a <em>quantum die</em>: when you roll it, the universe <em>splits into multiple copies</em>, one copy for each possible outcome of the die. In this case, rolling the die always splits the universe into <em>three copies</em>: one where the outcome of the roll was <code>1</code>, one where it was <code>2</code>, and one where it was <code>3</code>.</p>
<p>The game is played the same as before, although to prevent things from getting too far out of hand, the game now ends when either player's score reaches at least <code><em>21</em></code>.</p>
<p>Using the same starting positions as in the example above, player 1 wins in <code><em>444356092776315</em></code> universes, while player 2 merely wins in <code>341960390180808</code> universes.</p>
<p>Using your given starting positions, determine every possible outcome. <em>Find the player that wins in more universes; in how many universes does that player win?</em></p>
</article>

</main>


In [89]:
from functools import cache


def qantumDiracDice(player_1: int, player_2: int) -> bool:
    @cache
    def dfs(steps, player_1, player_2, score_1, score_2) -> tuple[int, int]:
        if score_1 >= 21:
            return 1, 0
        if score_2 >= 21:
            return 0, 1

        win_1, win_2 = 0, 0
        for roll_sum, count in [(3, 1), (4, 3), (5, 6), (6, 7), (7, 6), (8, 3), (9, 1)]:
            if steps & 1 == 0:
                pos = (player_1 + roll_sum) % 10
                player_11 = pos
                player_22 = player_2
                score_11 = score_1 + pos + 1
                score_22 = score_2
            else:
                pos = (player_2 + roll_sum) % 10
                player_11 = player_1
                player_22 = pos
                score_11 = score_1
                score_22 = score_2 + pos + 1

            w_1, w_2 = dfs(steps + 1, player_11, player_22, score_11, score_22)
            win_1, win_2 = win_1 + count * w_1, win_2 + count * w_2

        return win_1, win_2

    return dfs(0, player_1 - 1, player_2 - 1, 0, 0)


def max_winner(player_1: int, player_2: int) -> int:
    return max(qantumDiracDice(player_1, player_2))


assert qantumDiracDice(4, 8) == (444356092776315, 341960390180808)

assert max_winner(4, 8) == 444356092776315

In [90]:
from functools import cache


def qantumDiracDice(player_1: int, player_2: int) -> bool:
    @cache
    def dfs(
        steps: int,
        player_1: int,
        player_2: int,
        score_1: int,
        score_2: int,
    ) -> tuple[int, int]:
        if score_1 >= 21:
            return 1, 0
        if score_2 >= 21:
            return 0, 1

        win_1, win_2 = 0, 0
        for roll_sum, count in [(3, 1), (4, 3), (5, 6), (6, 7), (7, 6), (8, 3), (9, 1)]:
            if steps & 1 == 0:
                pos = (player_1 + roll_sum) % 10
                player_11 = pos
                player_22 = player_2
                score_11 = score_1 + pos + 1
                score_22 = score_2
            else:
                pos = (player_2 + roll_sum) % 10
                player_11 = player_1
                player_22 = pos
                score_11 = score_1
                score_22 = score_2 + pos + 1

            w_1, w_2 = dfs(steps + 1, player_11, player_22, score_11, score_22)
            win_1, win_2 = win_1 + count * w_1, win_2 + count * w_2

        return win_1, win_2

    return dfs(0, player_1 - 1, player_2 - 1, 0, 0)


def max_winner(player_1: int, player_2: int) -> int:
    return max(qantumDiracDice(player_1, player_2))


assert qantumDiracDice(4, 8) == (444356092776315, 341960390180808)

assert max_winner(4, 8) == 444356092776315

In [91]:
from functools import cache


def qantumDiracDice(player_1: int, player_2: int) -> bool:
    @cache
    def dfs(
        steps: int,
        player_1: int,
        player_2: int,
        score_1: int,
        score_2: int,
    ) -> tuple[int, int]:
        if score_1 >= 21:
            return 1, 0
        if score_2 >= 21:
            return 0, 1

        win_1, win_2 = 0, 0
        for roll_sum, count in [(3, 1), (4, 3), (5, 6), (6, 7), (7, 6), (8, 3), (9, 1)]:
            if steps & 1 == 0:
                pos = (player_1 + roll_sum) % 10
                player_11 = pos
                player_22 = player_2
                score_11 = score_1 + pos + 1
                score_22 = score_2
            else:
                pos = (player_2 + roll_sum) % 10
                player_11 = player_1
                player_22 = pos
                score_11 = score_1
                score_22 = score_2 + pos + 1

            w_1, w_2 = dfs(steps + 1, player_11, player_22, score_11, score_22)
            win_1, win_2 = win_1 + count * w_1, win_2 + count * w_2

        return win_1, win_2

    return dfs(0, player_1 - 1, player_2 - 1, 0, 0)


def max_winner(player_1: int, player_2: int) -> int:
    return max(qantumDiracDice(player_1, player_2))


assert qantumDiracDice(4, 8) == (444356092776315, 341960390180808)

assert max_winner(4, 8) == 444356092776315

In [92]:
from functools import cache


def qantumDiracDice(player_1: int, player_2: int) -> bool:
    @cache
    def dfs(steps, player_1, player_2, score_1, score_2) -> tuple[int, int]:
        if score_1 >= 21:
            return 1, 0
        if score_2 >= 21:
            return 0, 1

        win_1, win_2 = 0, 0
        for roll_sum, count in [(3, 1), (4, 3), (5, 6), (6, 7), (7, 6), (8, 3), (9, 1)]:
            if steps & 1 == 0:
                pos = (player_1 + roll_sum) % 10
                player_11 = pos
                player_22 = player_2
                score_11 = score_1 + pos + 1
                score_22 = score_2
            else:
                pos = (player_2 + roll_sum) % 10
                player_11 = player_1
                player_22 = pos
                score_11 = score_1
                score_22 = score_2 + pos + 1

            w_1, w_2 = dfs(steps + 1, player_11, player_22, score_11, score_22)
            win_1, win_2 = win_1 + count * w_1, win_2 + count * w_2

        return win_1, win_2

    return dfs(0, player_1 - 1, player_2 - 1, 0, 0)


def max_winner(player_1: int, player_2: int) -> int:
    return max(qantumDiracDice(player_1, player_2))


assert qantumDiracDice(4, 8) == (444356092776315, 341960390180808)

assert max_winner(4, 8) == 444356092776315

In [93]:
print(f"Part II: { max_winner(6, 7) }")

Part II: 911090395997650


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>911090395997650</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
